# MCS MEA — Pair Viewer (Inline Only)

HERE ARE THE GUI RELEVANT CHANGES

Instructions:
- Use the mcs_mea_env kernel (PyQt5 + pyqtgraph installed).
- Run the next cell to enable the Qt event loop in Jupyter.
- Run "Build Ready + Pairs".
- Set plate/pair/channel and run "Launch GUI".

Notes:
- Raw traces may be blank if raw H5 cannot be read in-notebook; IFR always shows.
- Chemical timestamps (if present) are shown as red dashed lines.
- Accepted/rejected channel selections save under `mcs_mea_outputs/selections/`.


In [2]:
from pathlib import Path
import h5py

h5_dir = Path("/Volumes/Manny2TB/mea_blade_round5_led_ctz/h5_files")

if not h5_dir.exists():
    raise SystemExit(f"Missing directory: {h5_dir}")

h5_files = sorted(h5_dir.glob("*.h5"))
print("FOUND_H5_FILES:", len(h5_files))
if not h5_files:
    raise SystemExit("No .h5 files found")

# Pick the first file (or change index)
h5_path = h5_files[0]
print("USING:", h5_path)

with h5py.File(h5_path, "r") as f:
    # Locate InfoChannel
    ds = f["Data/Recording_0/AnalogStream/Stream_0/InfoChannel"]
    fields = ds.dtype.names
    print("INFOCHANNEL FIELDS:", fields)

    ch_idx = 0  # change as needed
    row = ds[ch_idx]

    adzero = row["ADZero"]
    conv = row["ConversionFactor"]
    exp = row["Exponent"]
    unit = row["Unit"]

    print(f"\nCHANNEL {ch_idx}:")
    print("ADZero:", adzero)
    print("ConversionFactor:", conv)
    print("Exponent:", exp)
    print("Unit:", unit)

    # Scaling factor (volts per count)
    volts_per_count = conv * (10 ** exp)

    # Convert to microvolts per count
    uv_per_count = volts_per_count * 1e6

    print("\nSCALE:")
    print("volts/count =", volts_per_count)
    print("µV/count    =", uv_per_count)

    # Verify conversion on example raw values
    test_counts = [1000, 2000, 10000, -4000]
    print("\nEXAMPLE CONVERSIONS:")
    for raw in test_counts:
        volts = (raw - adzero) * volts_per_count
        uv = volts * 1e6
        print(f"raw={raw:>6} -> {uv:>10.3f} µV  ({volts:.6e} V)")


FOUND_H5_FILES: 12
USING: /Volumes/Manny2TB/mea_blade_round5_led_ctz/h5_files/plate_01_led_ctz_2023-12-05T08-55-49.h5
INFOCHANNEL FIELDS: ('ChannelID', 'RowIndex', 'GroupID', 'ElectrodeGroup', 'Label', 'RawDataType', 'Unit', 'Exponent', 'ADZero', 'Tick', 'ConversionFactor', 'ADCBits', 'HighPassFilterType', 'HighPassFilterCutOffFrequency', 'HighPassFilterOrder', 'LowPassFilterType', 'LowPassFilterCutOffFrequency', 'LowPassFilterOrder')

CHANNEL 0:
ADZero: 0
ConversionFactor: 59605
Exponent: -12
Unit: b'V'


ValueError: Integers to negative integer powers are not allowed.

In [ ]:
%gui qt5
import os
os.environ.setdefault('QT_MAC_WANTS_LAYER','1')

In [ ]:
# Build Ready + Pairs (self-contained)
import sys
from pathlib import Path
import pandas as pd
# Ensure repo root on sys.path
def _ensure_repo_on_path():
    here = Path.cwd()
    for cand in [here, *here.parents]:
        if (cand / 'mcs_mea_analysis').exists():
            if str(cand) not in sys.path:
                sys.path.insert(0, str(cand))
            return cand
    return here
REPO_ROOT = _ensure_repo_on_path()
from pathlib import Path as _P
from mcs_mea_analysis.ready import ReadinessConfig, build_ready_index
from mcs_mea_analysis.pairings import PairingIndex
OUTPUT_ROOT = _P('/Volumes/Manny2TB/mcs_mea_outputs') if (_P('/Volumes/Manny2TB/mcs_mea_outputs').exists()) else _P('_mcs_mea_outputs_local')
ready_csv, _, ready_rows = build_ready_index(ReadinessConfig(output_root=OUTPUT_ROOT, require_ifr_npz=True))
df = pd.DataFrame(ready_rows)
px = PairingIndex.from_ready_rows(df.to_dict('records'), group_by_round=True)
pairs_df = pd.DataFrame(px.pairs_dataframe())
ready_pairs_df = pairs_df.query('pair_status=="ready_pair"').copy()
print('Ready pairs:', len(ready_pairs_df))
print('Pairs per plate:')
display(ready_pairs_df.groupby('plate').size().to_frame('n_pairs'))


In [ ]:
# Inline launcher function (only)
from mcs_mea_analysis.pair_viewer_gui import PairInputs, launch_pair_viewer
from pathlib import Path as _Path2
import pandas as pd

def _get_pair_context(plate:int, pair_idx:int=0):
    rpp = ready_pairs_df[ready_pairs_df['plate']==plate].reset_index(drop=True)
    if rpp.empty: raise RuntimeError(f'No ready pairs for plate {plate}')
    if not (0 <= pair_idx < len(rpp)): raise RuntimeError(f'pair_idx {pair_idx} out of range [0..{len(rpp)-1}]')
    row = rpp.iloc[pair_idx]
    ctz_h5 = df.loc[df['recording_stem']==row['ctz_stem'], 'path']
    veh_h5 = df.loc[df['recording_stem']==row['veh_stem'], 'path']
    chem_c = df.loc[df['recording_stem']==row['ctz_stem'], 'chem_timestamp']
    chem_v = df.loc[df['recording_stem']==row['veh_stem'], 'chem_timestamp']
    return row, ctz_h5, veh_h5, chem_c, chem_v

def open_pair_inline(plate:int, pair_idx:int=0, ch:int=0):
    row, ctz_h5, veh_h5, chem_c, chem_v = _get_pair_context(plate, pair_idx)
    pin = PairInputs(
        plate=int(row['plate']) if pd.notna(row['plate']) else None,
        round=str(row['round']) if pd.notna(row['round']) else None,
        ctz_npz=_Path2(row['ctz_npz_path']),
        veh_npz=_Path2(row['veh_npz_path']),
        ctz_h5=_Path2(ctz_h5.iloc[0]) if not ctz_h5.empty else None,
        veh_h5=_Path2(veh_h5.iloc[0]) if not veh_h5.empty else None,
        chem_ctz_s=float(chem_c.iloc[0]) if not chem_c.empty and pd.notna(chem_c.iloc[0]) else None,
        chem_veh_s=float(chem_v.iloc[0]) if not chem_v.empty and pd.notna(chem_v.iloc[0]) else None,
        initial_channel=int(ch),
    )
    launch_pair_viewer(pin)


In [ ]:
# Launch GUI — set inputs here
plate = 2# e.g., 4,5,2
pair_idx = 0 # 0-based index within that plate
ch = 0       # channel to start at
open_pair_inline(plate=plate, pair_idx=pair_idx, ch=ch)